In [1]:
import os
from pprint import pprint

import duckdb
import pandas as pd
import cfbd
from cfbd.rest import ApiException
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Get api key
api_key = os.getenv("CFB_KEY")

# Configure API key authorization: ApiKeyAuth
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = api_key


In [3]:
# Uncomment below to setup prefix (e.g. Bearer) for API key, if needed
configuration.api_key_prefix['Authorization'] = 'Bearer'

# create an instance of the API class with the drives api
api_instance = cfbd.DrivesApi(cfbd.ApiClient(configuration))

# set params for the data query
# year is only required param
# season type defaults to regular
# docs: https://api.collegefootballdata.com/api/docs/?url=/api-docs.json#/drives/getDrives

year = 2023 # int | Year filter
season_type = 'regular' # str | Season type filter (optional) (default to regular)
week = 1 # int | Week filter (optional)
team = 'Nebraska' # str | Team filter (optional)


try:
    # Drive data and results
    api_response = api_instance.get_drives(year, season_type=season_type, week=week, team=team)
    # pprint(api_response)
except ApiException as e:
    print("Exception when calling DrivesApi->get_drives: %s\n" % e)

[{'defense': 'Nebraska',
 'defense_conference': 'Big Ten',
 'drive_number': 1,
 'drive_result': 'PUNT',
 'end_defense_score': 0,
 'end_offense_score': 0,
 'end_period': 1,
 'end_time': {'minutes': 9, 'seconds': 45},
 'end_yardline': 47,
 'end_yards_to_goal': 53,
 'game_id': 401520148,
 'id': 4015201481,
 'is_home_offense': True,
 'offense': 'Minnesota',
 'offense_conference': 'Big Ten',
 'plays': 9,
 'scoring': False,
 'start_defense_score': 0,
 'start_offense_score': 0,
 'start_period': 1,
 'start_time': {'minutes': 14, 'seconds': 55},
 'start_yardline': 19,
 'start_yards_to_goal': 81,
 'yards': 28},
 {'defense': 'Minnesota',
 'defense_conference': 'Big Ten',
 'drive_number': 2,
 'drive_result': 'PUNT',
 'end_defense_score': 0,
 'end_offense_score': 0,
 'end_period': 1,
 'end_time': {'minutes': 5, 'seconds': 12},
 'end_yardline': 67,
 'end_yards_to_goal': 67,
 'game_id': 401520148,
 'id': 4015201482,
 'is_home_offense': False,
 'offense': 'Nebraska',
 'offense_conference': 'Big Ten',


In [4]:
print(len(api_response))

19


In [5]:
# Convert the list of Drive objects to a list of dictionaries
drives_data = [drive.to_dict() for drive in api_response]

In [6]:
# Create a DataFrame from the list of dictionaries
df_drives = pd.DataFrame(drives_data)

In [8]:
# Create a DuckDB connection (to a file)
conn = duckdb.connect('../CFB_DuckDB.db')

In [9]:
# Insert the DataFrame into a DuckDB table
conn.execute("CREATE TABLE drives AS SELECT * FROM df_drives")

In [10]:
# Close database connection
conn.close()